<a href="https://colab.research.google.com/github/sajithrw/RetinaInspect-AI/blob/develop/RetinaInspect_AI_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DataSet**

Using dataset *Diabetic Retinopathy 2015 Data Colored Resized* from kaggle.com. These images are resized into 224x224 pixels so that they can be readily used with many pre-trained deep learning models.

All of the images are already saved into their respective folders according to the severity/stage of diabetic retinopathy using the train.csv file provided. You will find five directories with the respective images:

0 - No_DR\
1 - Mild\
2 - Moderate\
3 - Severe\
4 - Proliferate_DR

The trainLabels.csv contains all the image ids and their corresponding severity.

In [2]:
import tensorflow as tf
print(tf.__version__)

2.13.0
